In [1]:
##create Virtual Folder Structure...
import girder_client
from girder_utils import recurseGetResource
import json
import pprint
API_URL = "http://adrc.digitalslidearchive.emory.edu:8080/api/v1"

gc = girder_client.GirderClient(apiUrl=API_URL)
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [2]:
##Let's get the project by Name
CollectionName = 'ADRC'
ADRCCollUID =  gc.resourceLookup('/collection/%s' % CollectionName)

## Let's also get the projectMetadata folder for this collection
metaDataFolder =  list(gc.listFolder(ADRCCollUID['_id'],parentFolderType='collection',name='.projectMetaData'))[0]
pprint.pprint(metaDataFolder)

{u'_accessLevel': 2,
 u'_id': u'5ab27f9592ca9a0017a2095b',
 u'_modelType': u'folder',
 u'baseParentId': u'596e3a4a92ca9a000b2af2a7',
 u'baseParentType': u'collection',
 u'created': u'2018-03-21T15:51:49.607000+00:00',
 u'creatorId': u'596e36eb92ca9a000b81da75',
 u'description': u'',
 u'meta': {u'projectSchema': {u'patientID': [u'A01-94',
                                             u'A03-138',
                                             u'A11-120',
                                             u'A12-62',
                                             u'A16-15',
                                             u'A16-24',
                                             u'A16-27',
                                             u'A16-33',
                                             u'A16-34',
                                             u'E04-125',
                                             u'E04-155',
                                             u'E04-48',
                                        

In [ ]:
# ## For the first iteration, I want to bootstrap this with the list of patient folders we have already created
# patientFolder = gc.listFolder('5a2039e592ca9a0018254053')

# patientIDList = [x['name'] for x in patientFolder]
# print patientIDList

In [3]:
### Be Careful you don't overwrite the projectScheme with whatever you manually edit here... unless you mean to do it

stainType = ["TDP","TAU","AB","SYN","UBIQ","BIEL","HE","SM","THIO","LFB","NF"]
patientID = [u'A01-94', u'A03-138', u'A11-120', u'A12-62', u'A16-15', u'A16-24', u'A16-27', u'A16-33', u'A16-34', u'E04-125', u'E04-155', u'E04-48', u'E04-76', u'E04-99', u'E05-100', u'E05-108', u'E05-194', u'E05-200', u'E06-06', u'E06-112', u'E06-160', u'E06-18', u'E06-52', u'E06-97', u'E07-48', u'E07-85', u'E08-119', u'E08-146', u'E08-162', u'E09-08', u'E09-155', u'E09-164', u'E09-35', u'E09-81', u'E09-91', u'E10-110', u'E10-129', u'E10-137', u'E10-150', u'E10-151', u'E10-160', u'E10-179', u'E10-38', u'E10-48', u'E10-56', u'E10-63', u'E10-81', u'E10-83', u'E10-88', u'E11-112', u'E11-124', u'E11-125', u'E11-128', u'E11-139', u'E11-143', u'E11-20', u'E11-41', u'E11-59', u'E11-60', u'E11-70', u'E11-71', u'E11-73', u'E11-75', u'E11-81', u'E11-83', u'E11-84', u'E11-97', u'E12-06', u'E12-110', u'E12-120', u'E12-14', u'E12-32', u'E12-64', u'E12-74', u'E12-85', u'E15-01', u'E15-101', u'E15-106', u'E15-110', u'E15-111', u'E15-125', u'E15-132', u'E15-140', u'E15-30', u'E15-45', u'E15-46', u'E15-92', u'E15-97', u'E16-11', u'E16-110', u'E16-114', u'E16-123', u'E16-128', u'E16-131', u'E16-138', u'E16-16', u'E16-18', u'E16-19', u'E16-21', u'E16-45', u'E16-66', u'E17-05', u'E17-12', u'E17-17', u'E17-21', u'E17-22', u'E17-32', u'E17-36', u'E17-39', u'E17-40', u'E17-47', u'OS00-10', u'OS00-11', u'OS00-14', u'OS00-19', u'OS00-21', u'OS00-40', u'OS00-41', u'OS01-02', u'OS01-06', u'OS01-10', u'OS01-134', u'OS01-152', u'OS01-33', u'OS01-60', u'OS03-300', u'OS03-385', u'OS03-95', u'OS160608', u'OS161018', u'OS86-04', u'OS89-44', u'OS93-24', u'OS96-33', u'OS97-60', u'OS97-61', u'OS99-04', u'OS99-07', u'OS_170124']

projectSchema = { 'stainType': stainType, 'patientID': patientID}
print projectSchema


{'patientID': [u'A01-94', u'A03-138', u'A11-120', u'A12-62', u'A16-15', u'A16-24', u'A16-27', u'A16-33', u'A16-34', u'E04-125', u'E04-155', u'E04-48', u'E04-76', u'E04-99', u'E05-100', u'E05-108', u'E05-194', u'E05-200', u'E06-06', u'E06-112', u'E06-160', u'E06-18', u'E06-52', u'E06-97', u'E07-48', u'E07-85', u'E08-119', u'E08-146', u'E08-162', u'E09-08', u'E09-155', u'E09-164', u'E09-35', u'E09-81', u'E09-91', u'E10-110', u'E10-129', u'E10-137', u'E10-150', u'E10-151', u'E10-160', u'E10-179', u'E10-38', u'E10-48', u'E10-56', u'E10-63', u'E10-81', u'E10-83', u'E10-88', u'E11-112', u'E11-124', u'E11-125', u'E11-128', u'E11-139', u'E11-143', u'E11-20', u'E11-41', u'E11-59', u'E11-60', u'E11-70', u'E11-71', u'E11-73', u'E11-75', u'E11-81', u'E11-83', u'E11-84', u'E11-97', u'E12-06', u'E12-110', u'E12-120', u'E12-14', u'E12-32', u'E12-64', u'E12-74', u'E12-85', u'E15-01', u'E15-101', u'E15-106', u'E15-110', u'E15-111', u'E15-125', u'E15-132', u'E15-140', u'E15-30', u'E15-45', u'E15-46', u'

In [4]:
### Be careful... updating the projectSchema here..
gc.addMetadataToFolder(metaDataFolder['_id'],{ 'projectSchema': projectSchema} )
## We may also want to keep other things here like TotalSlides, TotalPatients, TotalStains, whatever as top level parameters

{u'_accessLevel': 2,
 u'_id': u'5ab27f9592ca9a0017a2095b',
 u'_modelType': u'folder',
 u'baseParentId': u'596e3a4a92ca9a000b2af2a7',
 u'baseParentType': u'collection',
 u'created': u'2018-03-21T15:51:49.607000+00:00',
 u'creatorId': u'596e36eb92ca9a000b81da75',
 u'description': u'',
 u'meta': {u'projectSchema': {u'patientID': [u'A01-94',
    u'A03-138',
    u'A11-120',
    u'A12-62',
    u'A16-15',
    u'A16-24',
    u'A16-27',
    u'A16-33',
    u'A16-34',
    u'E04-125',
    u'E04-155',
    u'E04-48',
    u'E04-76',
    u'E04-99',
    u'E05-100',
    u'E05-108',
    u'E05-194',
    u'E05-200',
    u'E06-06',
    u'E06-112',
    u'E06-160',
    u'E06-18',
    u'E06-52',
    u'E06-97',
    u'E07-48',
    u'E07-85',
    u'E08-119',
    u'E08-146',
    u'E08-162',
    u'E09-08',
    u'E09-155',
    u'E09-164',
    u'E09-35',
    u'E09-81',
    u'E09-91',
    u'E10-110',
    u'E10-129',
    u'E10-137',
    u'E10-150',
    u'E10-151',
    u'E10-160',
    u'E10-179',
    u'E10-38',
    u'E1

In [5]:
### Going to now go through the projectSchema and create a virtual folder structure, in this first example
## I am creating a folder structure based on patientID-->stainType
## We may also do patientID-->Region-->StainType or other iterations in other examples below
##Let's get the project by Name
CollectionName = 'ADRC.VirtualFolderStructure'
virtualADRCCollUID =  gc.resourceLookup('/collection/%s' % CollectionName)

In [11]:
## First create it by Patient and then by Stain
TopLevelFolderName = "ByPatient"
TLF = gc.createFolder(virtualADRCCollUID['_id'],TopLevelFolderName,parentType='collection',reuseExisting=True)

## We now create a folder for every patient, and then a separate folder by stain
for pt in projectSchema['patientID']:
    ## Now create a virtual folder for each patient...
    virtualPatientFolder  = gc.createFolder(TLF['_id'],pt,description=pt,reuseExisting=True)
    
    for stain in projectSchema['stainType']:
        params = { "parentType": "folder",
               "parentId"  : virtualPatientFolder['_id'],
               "reuseExisting": True,
               "description" : stain,
               "name":         stain,
                "isVirtual": True,
                "virtualItemsQuery": json.dumps({"meta.stainType": stain,  "meta.patientID": pt, 'baseParentId': {"$oid": ADRCCollUID['_id']}})
                 }
        virtualStainFolder =  gc.post("folder",parameters=params)
    ## CREATE AN ALL FOLDER FOR EVERYTHING
    params = { "parentType": "folder",
           "parentId"  : virtualPatientFolder['_id'],
           "reuseExisting": True,
           "description" : 'ALL',
           "name":         'ALL',
            "isVirtual": True,
           "virtualItemsQuery": json.dumps({ "meta.patientID": pt})
             }
    anyPtSlide =  gc.post("folder",parameters=params)
## As a cleanup step, I need to delete virtualFolders that contain 0 virtualItems, as it's annoying
## alhttp://localhost:8890/notebooks/createADRCVirtualFolderStructure.ipynb#though we could also change the UI to not render them ... TBD

In [ ]:
#### Let's look for empty folders...
virtualPtFolderList =  list( gc.listFolder(TLF['_id'],parentFolderType='folder') )

for vpf in virtualPtFolderList:
    ### Let's now see what's in each of these folders
    ptByStainFolders = gc.listFolder(vpf['_id'])
    for pbsf in ptByStainFolders:
        itemsInCurFolder =  list(gc.listItem(pbsf['_id']))
        print len(itemsInCurFolder),pbsf['name'],vpf['name']
#         if len(itemsInCurFolder) == 0:
#             gc.delete('/folder/%s'%pbsf['_id'])
#             print "Deleting %s for pt %s which has %d items" % ( pbsf['name'],vpf['name'], len(itemsInCurFolder))


0 AB A01-94
0 ALL A01-94
0 BIEL A01-94
0 HE A01-94
0 LFB A01-94
0 NF A01-94
0 SM A01-94
0 SYN A01-94
0 TAU A01-94
0 TDP A01-94
0 THIO A01-94
0 UBIQ A01-94
0 AB A03-138
0 ALL A03-138
0 BIEL A03-138
0 HE A03-138
0 LFB A03-138
0 NF A03-138
0 SM A03-138
0 SYN A03-138
0 TAU A03-138
0 TDP A03-138
0 THIO A03-138
0 UBIQ A03-138
0 AB A11-120
0 ALL A11-120
0 BIEL A11-120
0 HE A11-120
0 LFB A11-120
0 NF A11-120
0 SM A11-120
0 SYN A11-120
0 TAU A11-120
0 TDP A11-120
0 THIO A11-120
0 UBIQ A11-120
0 AB A12-62
0 ALL A12-62
0 BIEL A12-62
0 HE A12-62
0 LFB A12-62
0 NF A12-62
0 SM A12-62
0 SYN A12-62
0 TAU A12-62
0 TDP A12-62
0 THIO A12-62
0 UBIQ A12-62
22 AB A16-15
144 ALL A16-15
22 BIEL A16-15
48 HE A16-15
2 LFB A16-15
0 NF A16-15
0 SM A16-15
0 SYN A16-15
0 TAU A16-15
34 TDP A16-15
0 THIO A16-15
0 UBIQ A16-15
16 AB A16-24
138 ALL A16-24
2 BIEL A16-24
60 HE A16-24
8 LFB A16-24
0 NF A16-24
0 SM A16-24
0 SYN A16-24
0 TAU A16-24
42 TDP A16-24
0 THIO A16-24
0 UBIQ A16-24
26 AB A16-27
150 ALL A16-27
8 BIEL 

In [ ]:
## Organize by Region MetaData Key
## Folder Name is VF.ByRegion
ByRegionFolderID = '5a94567b92ca9a00186a2f4d'


#folder?parentType=folder&parentId=5a94567b92ca9a00186a2f4d&name=Cerebellum&description=Cerebellum%20slides&reuseExisting=true&isVirtual=true&virtualItemsQuery=%7B%22meta.brainRegion%22%3A%22Cerebellum%22%7D
regionList = ["Amygdala","BasalGanglia","BasalGanglia_NucleusBasalis","Cerebellum","CingulateCortex","FrontalCortex",
"Hippocampus","InsularCortex","Medulla","OccipitalCortex","ParietalCortex","PeriRolandic_eight","PeriRolandic_seven","Pons","TemporalCortex"]

for regionName in regionList:
    
    folderGenQuery=  "{\"meta.brainRegion\":\"%s\"}" % regionName
    postRoute = "folder?parentType=folder&parentId=%s&name=%s&description=%s&reuseExisting=true&isVirtual=true&virtualItemsQuery=%s"
    postRoute_wp = postRoute % ( ByRegionFolderID, regionName, regionName, folderGenQuery)
    print regionName
    gc.post(postRoute_wp)
